<a href="https://colab.research.google.com/github/sunil-karki/Google3QLA-col/blob/master/Google3QLA_COLAB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train (1).csv


In [0]:
import io
import pandas as pd
X_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
# Dataset is now stored in a Pandas Dataframe

#X_train = pd.read_csv("../input/google-quest-challenge/train.csv")
print( len(X_train.columns), ' ', len(X_train))

41   6079


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
!pip install kaggle

In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c google-quest-challenge 

  0% 0.00/954k [00:00<?, ?B/s]
100% 954k/954k [00:00<00:00, 64.4MB/s]
  0% 0.00/4.50M [00:00<?, ?B/s]
100% 4.50M/4.50M [00:00<00:00, 148MB/s]
  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 92.3MB/s]


In [10]:
!ls

kaggle_1.json  sample_data	      test.csv
kaggle.json    sample_submission.csv  train.csv.zip


In [0]:
import pandas as pd
#df = pd.read_csv('filename.zip', compression='zip', header=0, sep=',', quotechar='"')
X_train = pd.read_csv('train.csv.zip', compression='zip')

#X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
#        'category','host']]

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [13]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
#Imports
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

from keras import optimizers

from sklearn.model_selection import train_test_split

In [15]:
X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
        'category','host']]

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com
...,...,...,...,...,...,...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,sixtyfootersdude,https://bicycles.stackexchange.com/users/134,If you're thinking about wearing a ski helmet ...,Matt Leo,https://bicycles.stackexchange.com/users/3340,http://bicycles.stackexchange.com/questions/99...,CULTURE,bicycles.stackexchange.com
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,ash,https://bicycles.stackexchange.com/users/14519,\nYou can replace the pads (as stated elsewher...,Daniel R Hicks,https://bicycles.stackexchange.com/users/1584,http://bicycles.stackexchange.com/questions/25...,CULTURE,bicycles.stackexchange.com
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maneating Koala,https://unix.stackexchange.com/users/60445,Maybe help if can be fixes origin of this erro...,BG Bruno,https://unix.stackexchange.com/users/68208,http://unix.stackexchange.com/questions/169054...,TECHNOLOGY,unix.stackexchange.com
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,MrB,https://mathoverflow.net/users/2189,"As a non-mathematician, I am somewhat mystifie...",angela,https://mathoverflow.net/users/4267,http://mathoverflow.net/questions/57337,SCIENCE,mathoverflow.net


In [16]:
X_train[['qa_id','question_title','question_body','answer','category']]

,qa_id,question_title,question_body,answer,category
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin...",LIFE_ARTS
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...,CULTURE
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...,SCIENCE
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between...",CULTURE
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...,LIFE_ARTS
...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,If you're thinking about wearing a ski helmet ...,CULTURE
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,\nYou can replace the pads (as stated elsewher...,CULTURE
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maybe help if can be fixes origin of this erro...,TECHNOLOGY
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,"As a non-mathematician, I am somewhat mystifie...",SCIENCE


In [0]:
#X_train['question_body'][0]
##  https://www.figure-eight.com/data-for-everyone/

In [0]:
# 'question_title','question_body','category' to be added together here
X_train['question_body'] = X_train['question_body'] + ' ' + X_train['question_title'] + ' ' + X_train['category']
del X_train['question_title']
del X_train['category']
#X_train['category']

In [19]:
X_train.columns

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host',
       'question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
 

In [20]:
import nltk
#Resource stopwords and wordnet not found
#needed to do this below in colab
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#import nltk
import re

stopword = nltk.corpus.stopwords.words('english')

lm = nltk.WordNetLemmatizer()
st = nltk.PorterStemmer()

class NLP_preprocess:
    def tokenize(self, text):
        tokens = re.split('\W+', text)
        return tokens

    def remove_stopwords(self, tokenized_text):
        text = [word for word in tokenized_text if word not in stopword]
        return text
    
    def lemmatizing(self, text):
        text = [lm.lemmatize(word) for word in text]
        return text
    
    def stemming(self, text):
        text = [st.stem(word) for word in text]
        return text

    def process_column(self, col):
        col = col.apply(lambda x: self.tokenize(x.lower()))
        col = col.apply(lambda x: self.remove_stopwords(x))
        col = col.apply(lambda x: self.lemmatizing(x))
        return col


In [0]:
nlp = NLP_preprocess()

#Contains column which are text. Needs to be converted to Token
list_col = ['answer', 'question_body']

for col in list_col:
    X_train[col] = nlp.process_column(X_train[col])

In [0]:
class NLP_columns:
    def join_col(self, list_col):
        new_col = X_train[list_col[0]]
    
        for col in list_col[1:]:
            new_col += X_train[col]
        
        return new_col

    #Removing unnecessary elements from the processes joined Tokenized column
    def remove_values_from_list(self, the_list, val):
        return [value for value in the_list if value != val]
    
    def remove_values_from_column(self, col, val):
        return col.apply(lambda row: self.remove_values_from_list(row, val))


In [24]:
'''
def process_vocab():
    vocab = set()
    for a in X_train['answer']:
        vocab = vocab.union(set(a))

    for a in X_train['question_body']:
        vocab = vocab.union(set(a))
    
    vocab_len = len(vocab)
    #print(vocab_len)      #49474
    return vocab_len
    
vocab_len = process_vocab()
print(vocab_len)
'''

"\ndef process_vocab():\n    vocab = set()\n    for a in X_train['answer']:\n        vocab = vocab.union(set(a))\n\n    for a in X_train['question_body']:\n        vocab = vocab.union(set(a))\n    \n    vocab_len = len(vocab)\n    #print(vocab_len)      #49474\n    return vocab_len\n    \nvocab_len = process_vocab()\nprint(vocab_len)\n"

In [0]:
#print(vocab)
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

y = X_train[ylabels].astype(np.int64)



In [0]:
#Delete unwanted columns from X_train
#del X_train['question_asker_intent_understanding']
#X_train.drop(['question_asker_intent_understanding', 'question_body_critical'], axis=1, inplace=True)
X_train.drop(ylabels, axis=1, inplace=True)

In [27]:
X_train.columns     #Will need only  'answer' 'question_body'

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host'],
      dtype='object')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.33, random_state=np.random, shuffle=True)

In [29]:
print(X_test.shape)

(2007, 9)


In [30]:
'''
from sklearn.model_selection import train_test_split
X, y = np.arange(20).reshape((10, 2)), range(10)
print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=np.random, shuffle=True)
print(X_train)
print(y_train)
'''

'\nfrom sklearn.model_selection import train_test_split\nX, y = np.arange(20).reshape((10, 2)), range(10)\nprint(X)\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=np.random, shuffle=True)\nprint(X_train)\nprint(y_train)\n'

In [0]:
class NLP_before_model:
    tokenizer = 0
    max_question_body_len = 0
    max_answers_seq_len = 0
    train_question_body = []
    train_answers = []
    
    x_answer = 0
    x_question = 0
    
    vocab = set()
    vocab_len = 0
    
    def __init__(self, x_answer, x_question):
        self.x_answer = x_answer
        self.x_question = x_question
        
        self.process_vocab()
        self.tokenizer = Tokenizer(filters = [])
        self.tokenizer.fit_on_texts(self.vocab)
        
        #self.process_for_model()
        
    def process_vocab(self):
        #vocab = set()
        for a in self.x_answer:
            self.vocab = self.vocab.union(set(a))

        for a in self.x_question:
            self.vocab = self.vocab.union(set(a))

        self.vocab_len = len(self.vocab)
        #print(vocab_len)      #49474
        #return self.vocab_len
        
    def process_for_model(self):
        for a in self.x_answer:
            self.train_answers.append(a)

        for a in self.x_question:
            self.train_question_body.append(a)
            
        a, b = self.process_vectorize(self.train_answers, self.train_question_body)
        return a, b
                    
    def process_vectorize(self, train_answers, train_question_body):
        
        train_answers_seq = self.tokenizer.texts_to_sequences(train_answers)
        train_question_body_seq = self.tokenizer.texts_to_sequences(train_question_body)

        self.max_question_body_len = max([len(data) for data in train_question_body_seq])        
        self.max_answers_seq_len = max([len(data) for data in train_answers_seq])
        
        input_answer = pad_sequences(train_answers_seq, maxlen = self.max_answers_seq_len)
        input_question = pad_sequences(train_question_body_seq, maxlen = self.max_question_body_len)

        return input_answer, input_question



In [0]:
nlpbefore_mdl = NLP_before_model(X_train['answer'], X_train['question_body'])

In [33]:
#Vectorize trrain data
train_input_answer, train_input_question = nlpbefore_mdl.process_for_model()   #To vectorize only
train_input_answer

array([[    0,     0,     0, ...,  3388,  6935,  2891],
       [    0,     0,     0, ..., 18286, 15314,  1624],
       [    0,     0,     0, ...,  7949,  3281,  2422],
       ...,
       [    0,     0,     0, ...,  8578, 32133, 24924],
       [    0,     0,     0, ...,  6869, 32414,  8645],
       [    0,     0,     0, ...,   742, 35514, 11705]], dtype=int32)

In [34]:
max_answers_seq_len = nlpbefore_mdl.max_answers_seq_len
max_question_body_len = nlpbefore_mdl.max_question_body_len
vocab_len = nlpbefore_mdl.vocab_len
print(max_answers_seq_len)
print(max_question_body_len)

1346
1832


In [35]:
#Vectorize test data
test_input_answer, test_input_question = nlpbefore_mdl.process_vectorize(X_test['answer'], X_test['question_body'])
test_input_answer

array([[    0,     0,     0, ..., 12066,  6587, 16872],
       [    0,     0,     0, ...,  3866, 20833,  1441],
       [    0,     0,     0, ...,  4560, 14593, 25704],
       ...,
       [    0,     0,     0, ..., 13489,  8182, 15958],
       [    0,     0,     0, ..., 32996, 17089, 34325],
       [    0,     0,     0, ..., 33131, 37362, 13178]], dtype=int32)

In [36]:
'''
max_answers_seq_len = nlpbefore_mdl.max_answers_seq_len
max_question_body_len = nlpbefore_mdl.max_question_body_len
vocab_len = nlpbefore_mdl.vocab_len
print(max_answers_seq_len)
print(max_question_body_len)
'''

'\nmax_answers_seq_len = nlpbefore_mdl.max_answers_seq_len\nmax_question_body_len = nlpbefore_mdl.max_question_body_len\nvocab_len = nlpbefore_mdl.vocab_len\nprint(max_answers_seq_len)\nprint(max_question_body_len)\n'

In [0]:
#vocablen = nlpbefore_mdl.process_vocab()
#print(vocablen)

In [38]:
print(max_answers_seq_len)
print(max_question_body_len)
print(vocab_len)

1346
1832
40405


In [0]:
class NLP_model():
    max_answers_seq_len = 0
    max_question_body_len = 0
    vocab_len = 0
    
    model = 0
    history = 0
    a = 0
    
    
    def __init__(self, max_answers_seq_len, max_question_body_len, vocab_len):
        self.max_answers_seq_len = max_answers_seq_len
        self.max_question_body_len = max_question_body_len
        self.vocab_len = vocab_len
    
    def create_model(self):
        input_sequence = Input((self.max_answers_seq_len,)) #As we dont know batch size yet
        question = Input((self.max_question_body_len,))

        #Create input encoder M
        input_encoder_m = Sequential()
        input_encoder_m.add(Embedding(input_dim = self.vocab_len,output_dim = 64)) #From paper
        input_encoder_m.add(Dropout(0.3))

        #Create input encoder C:
        input_encoder_c = Sequential()
        input_encoder_c.add(Embedding(input_dim = self.vocab_len,output_dim = self.max_question_body_len)) #From paper
        input_encoder_c.add(Dropout(0.3))

        #Create question encoder:
        question_encoder = Sequential()
        question_encoder.add(Embedding(input_dim = self.vocab_len,output_dim = 64,input_length = self.max_question_body_len)) #From paper
        question_encoder.add(Dropout(0.3))

        input_encoded_m = input_encoder_m(input_sequence)
        input_encoded_c = input_encoder_c(input_sequence)
        question_encoded = question_encoder(question)

        match = dot([input_encoded_m,question_encoded], axes = (2,2))
        match = Activation('softmax')(match)

        response = add([match,input_encoded_c])
        response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

        answer = concatenate([response, question_encoded])

        # Reduce the answer tensor with a RNN (LSTM)
        answer = LSTM(32)(answer)

        #Regularization with dropout:
        answer = Dropout(0.5)(answer)
        #Output layer:
        answer = Dense(30)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

        #Now we need to output a probability distribution for the vocab, using softmax:
        answer = Activation('softmax')(answer)

        #Now we build the final model:
        self.model = Model([input_sequence,question], answer)

    def compile_model(self):
        ##sgd = optimizers.SGD(lr=0.2, decay=1e-5, momentum=0.7, nesterov=True)  #0.01

        ##self.model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
        
        #model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        # model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #Categorical instead of binary cross entropy as because of the way we are training
        #we could actually see any of the words from the vocab as output
        #however, we should only see yes or no

        ## self.model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy']) #acc 6%
        self.model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy']) #accuracy around 75%
        
    def summary_model(self):
        self.model.summary()
        
    def fit_model(self, input_labels, input_answer, input_question):
        #self.a = input_labels.astype(np.int64)
        #input_train == answeer  answert train == label // label left tot do
        #  history = model.fit([inputs_train,questions_train],a, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))
        
        #print(type(self.a))
        #self.history = self.model.fit([input_answer,input_question], a, batch_size = 50, epochs = 1000)
        
        #self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1500)
        
        self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1000)
        #xxxxx
        
    def eval_model(self, answer, question):
        pred_results = self.model.predict(([answer,question]))
        
    def save_model(self):
        filename = 'google3qla.h5'
        self.model.save(filename)
        

In [43]:
nlpmodel = NLP_model(max_answers_seq_len, max_question_body_len, vocab_len)
nlpmodel.create_model()
nlpmodel.compile_model()
nlpmodel.summary_model()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1346)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1832)         0                                            
__________________________________________________________________________________________________
sequential_4 (Sequential)       multiple             2585920     input_3[0][0]                    
__________________________________________________________________________________________________
sequential_6 (Sequential)       (None, 1832, 64)     2585920     input_4[0][0]                    
____________________________________________________________________________________________

In [0]:
nlpmodel.fit_model(y_train, train_input_answer, train_input_question)

Epoch 1/1000
4072/4072 [==============================] - 537s 132ms/step - loss: 0.7473 - acc: 0.7626
Epoch 2/1000
4072/4072 [==============================] - 533s 131ms/step - loss: 0.7130 - acc: 0.7626
Epoch 3/1000
3375/4072 [=======================>......] - ETA: 1:31 - loss: 0.6958 - acc: 0.7638

In [0]:
#print(train_input_answer.shape)
nlpmodel.save_model()

In [0]:
from keras.models import load_model
ld_model = load_model('google3qla.h5')

In [0]:

y = np.zeros([2007, train_input_answer.shape[1] - test_input_answer.shape[1] ], dtype = int) #2007 is number of rows and 742 is column needed to match with train_answer
test_input_answer_1 = np.append(test_input_answer, y, axis=1)

print(test_input_answer_1.shape)
print(train_input_answer.shape)

y = np.zeros([2007, train_input_question.shape[1] - test_input_question.shape[1] ], dtype = int) #2007 is number of rows and 390 is column needed to match with train_question
test_input_question_1 = np.append(test_input_question, y, axis=1)

print(test_input_question_1.shape)
print(train_input_question.shape)

score = ld_model.evaluate([test_input_answer_1, test_input_question_1], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
print(y_test.shape)
print(test_input_answer.shape)
print(train_input_answer.shape)
print(type(test_input_answer))

In [0]:
print(test_input_question.shape)
print(train_input_question.shape)
print(type(test_input_question))

In [0]:
'''
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
'''

In [0]:
###   Rough Work  ###

In [0]:
'''
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

a = X_train[ylabels].astype(np.int64)
nlpmodel.fit_model(a, input_answer, input_question)
#nlpmodel.fit_model(X_train[ylabels])
'''

In [0]:
'''
def process_model_ready():
    tokenizer = Tokenizer(filters = [])
    tokenizer.fit_on_texts(vocab)

    train_question_body = []
    train_answers = []

    for a in X_train['answer']:
        train_answers.append(a)

    for a in X_train['question_body']:
        train_question_body.append(a)

    train_answers_seq = tokenizer.texts_to_sequences(train_answers)
    train_question_body_seq = tokenizer.texts_to_sequences(train_question_body)

    max_question_body_len = max([len(data) for data in train_question_body_seq])
    print(max_question_body_len)

    max_answers_seq_len = max([len(data) for data in train_answers_seq])
    print(max_answers_seq_len)

    input_answer = pad_sequences(train_answers_seq, maxlen=max_answers_seq_len)
    input_question = pad_sequences(train_question_body_seq, maxlen=max_question_body_len)
    
    return input_answer

input_answer = process_model_ready()
'''

In [0]:
'''
input_sequence = Input((max_answers_seq_len,)) #As we dont know batch size yet
question = Input((max_question_body_len,))

#Create input encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim = 64)) #From paper
input_encoder_m.add(Dropout(0.3))

#Create input encoder C:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim = max_question_body_len)) #From paper
input_encoder_c.add(Dropout(0.3))

#Create question encoder:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim = 64,input_length=max_question_body_len)) #From paper
question_encoder.add(Dropout(0.3))

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

response = add([match,input_encoded_c])
response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

answer = concatenate([response, question_encoded])

# Reduce the answer tensor with a RNN (LSTM)
answer = LSTM(32)(answer)

#Regularization with dropout:
answer = Dropout(0.5)(answer)
#Output layer:
answer = Dense(2)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

#Now we need to output a probability distribution for the vocab, using softmax:
answer = Activation('softmax')(answer)

#Now we build the final model:
model = Model([input_sequence,question], answer)

from keras import optimizers
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)  #0.01

model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
#model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#Categorical instead of binary cross entropy as because of the way we are training
#we could actually see any of the words from the vocab as output
#however, we should only see yes or no

model.summary()

#X_train['answer_well_written']
#X_train.columns
a = X_train[['answer_well_written','question_well_written']].astype(np.int64)
a

ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

b = X_train[ylabels].astype(np.int64)
b

#input_train == answeer  answert train == label // label left tot do
#  history = model.fit([inputs_train,questions_train],a, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))
  history = model.fit([input_answer,input_question],a, batch_size = 50, epochs = 1000)
  
'''